In [1]:
import spacy
import sys
import gensim
import glob
import re
import sys
import os 
import pandas as pd
import numpy as np
from tqdm import tqdm

In [2]:
df = pd.read_csv("/home/nlp/Downloads/Dataset/questions.csv")

In [3]:
len(df)

404351

In [4]:
df["question1"] = df["question1"].apply(lambda x: unicode(str(x),"utf-8"))


In [ ]:
df["question2"] = df["question2"].apply(lambda x: unicode(str(x),"utf-8"))

In [ ]:
import gensim

questions = list(df['question1']) + list(df['question2'])

# tokenize
c = 0
for question in tqdm(questions):
    questions[c] = list(gensim.utils.tokenize(question, deacc=True, lower=True))
    c += 1

# train model
model = gensim.models.Word2Vec(questions, size=300, workers=16, iter=10, negative=20)

# trim memory
model.init_sims(replace=True)

# creta a dict 
w2v = dict(zip(model.index2word, model.syn0))
print "Number of tokens in Word2Vec:", len(w2v.keys())

# save model
model.save('data/3_word2vec.mdl')
model.save_word2vec_format('data/3_word2vec.bin', binary=True)

100%|██████████| 808702/808702 [00:42<00:00, 19117.86it/s]


In [ ]:
import spacy
nlp = spacy.load('en')

In [ ]:
vesc1 = [doc.vector in doc in nlp.pipe(df['question1'], n_threads = 50)]
vesc2 = [doc.vector in doc in nlp.pipe(df['question2'],n_threads=50)]
vesc1 = np.array(vesc1)
vesc2 = np.array(vesc2)


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

questions = list(df['question1']) + list(df['question2'])

In [ ]:
tfidf = TfidfVectorizer(questions)

In [ ]:
ft_ft=tfidf.fit_transform(questions)

In [ ]:
word2tfid = dict(zip(tfidf.get_feature_names(),tfidf.idf_))

In [ ]:
from sklearn.cluster import MiniBatchKMeans
n_cluser = 5
k_model = MiniBatchKMeans()

In [ ]:
Kmeans = k_model.fit(ft_ft)

In [ ]:
%time
kmeans_cluster = k_model.predict(ft_ft)
kmeans_distance = k_model.transform(ft_ft)


In [ ]:
sorted_centroid = Kmeans.cluster_centers_.argsort()[:,::-1]

In [ ]:
terms = tfidf.get_feature_names()